# COCO

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import pycocotools.mask as coco_mask
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [3]:
dataDir='../coco'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir, dataType)
coco=COCO(annFile)

loading annotations into memory...
Done (t=1.16s)
creating index...
index created!


In [4]:
# coco images
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
appliance outdoor indoor furniture kitchen person food accessory vehicle animal sports electronic


# Model

In [5]:
from tensorflow.keras.applications import inception_v3 as inc_net
from lime import lime_image
import requests as r
import xml.etree.ElementTree as ET
import numpy as np

from explanator import Explanator

Tensorflow 2.0.0
Keras 2.2.4-tf


In [6]:
inet_model = inc_net.InceptionV3()
explainer = lime_image.LimeImageExplainer()
root = ET.fromstring(r.get("http://www.image-net.org/api/xml/structure_released.xml").text)
imagenet_classes = eval(r.get("https://raw.githubusercontent.com/raghakot/keras-vis/master/resources/imagenet_class_index.json").text)

In [7]:
model = Explanator(inet_model=inet_model, explainer=explainer, root=root, imagenet_classes=imagenet_classes)

# Quantitative analysis - Cat

In [8]:
# drop watershed for this test
segmentation_fns = {
  'quickshift': {
      'kernel_size': 3,
      'max_dist': 200,
      'ratio': 0.2
  },
  'felzenszwalb': {
      'scale': 100,
      'sigma': 2,
      'min_size': 50,
  },
  'slic': {
      'compactness': 8,
      'n_segments': 1000,
      'sigma': 0.9
  },
#   'watershed': {
#       'markers': 100,
#       'compactness': 0.001
#   },
#   'canny': { # to-do: not supported
#       'min_size': 10,
#       'sigma': 1.8
#   },
#   'morphological_geodesic_active_contour': { # to-do: not supported
#       'iterations': 200,
#       'smoothing': 1,
#       'balloon' : -1,
#       'threshold' : 0.8
#   }
}

In [9]:
category = ["cat"]
catIds = coco.getCatIds(catNms=category);
imgIds = coco.getImgIds(catIds=catIds);
print(f"Coco images with category {category[0]}: {len(imgIds)}")

Coco images with category cat: 184


In [ ]:
%%time
results = {}
for i in imgIds[1:70]:
    print(i)
    # easy example 524280
    img_md = coco.loadImgs(i)[0]
    im, x = model.download_and_preprocess(img_md['coco_url'])
    print(model.predict(x))
    result = model.explain(x[0], img_md, segmentation_fns, coco, top_labels=1000, num_samples=1000)
    results[i] = result

269314
[('n07753592', 'banana', 0.8107299), ('n03355925', 'flagpole', 0.07722583), ('n03944341', 'pinwheel', 0.006936441), ('n04346328', 'stupa', 0.0051602814), ('n01819313', 'sulphur-crested_cockatoo', 0.0037510993)]
Start explain_instance


In [ ]:
# results = model.explain(x[0], img_md, segmentation_fns, num_samples=100)
# results is a dictionary containing masks and metrics per segmentation and labels.
# it includes also a union mask with its metrics.

# Segmentation

In [ ]:
# easy example 524280
imgIds = coco.getImgIds(imgIds = [416256])
img_md = coco.loadImgs(imgIds[np.random.randint(0, len(imgIds))])[0]
# img_md = coco.loadImgs(imgIds)

im, x = model.download_and_preprocess(img_md['coco_url'])

plt.figure(figsize=(10, 10))
plt.axis("off")
plt.imshow(im)

In [ ]:
##### to-do: future refactoring -> add parameters to segmentation_fns dictionary. Check plot_segmentations method.
segmentation_fns = {
  'quickshift': {
      'kernel_size': 3,
      'max_dist': 200,
      'ratio': 0.2
  }, 
  'felzenszwalb': {
      'scale': 100,
      'sigma': 2,
      'min_size': 50,
  },
  'slic': {
#        'n_segments': 1000,
       'compactness': 8,
#        'sigma': 0.9
  },
  'watershed': {
      'markers': 100,
      'compactness': 0.001
  },
#   'canny': {
#       'min_size': 10,
#       'sigma': 1.8
#   },
#   'morphological_geodesic_active_contour': {
#       'iterations': 200,
#       'smoothing': 1,
#       'balloon' : -1,
#       'threshold' : 0.8
#   }
}

model.plot_segmentations([np.array(im)][0], segmentation_fns)